In [ ]:
#AES Encryption & Decryption Website
# AES-128, ECB Mode

import numpy as np
import json
from bottle import Bottle, request, response, run, template, static_file, redirect
from time import time

app = Bottle()

#generating S-box, following C program from link below
#https://en.wikipedia.org/wiki/Rijndael_S-box

def rotate(x,shift):
    #Rotate 8-bit integer `x` to the left by `shift` positions.
    return ((x << shift) | (x >> (8 - shift))) & 0xFF

def create_SBOX():  
    SBOX = [0] * 256
    p = 1
    q = 1
    
    while True:
        #multiply p by 3
        p ^= (p << 1) ^ (0x1B if p & 0x80 else 0)
        p &= 0xFF
        
        #divide q by 3 (equals multiplication by 0xf6)
        q ^= (q << 1) & 0xFF
        q ^= (q << 2) & 0xFF
        q ^= (q << 4) & 0xFF
        q ^= (0x09 if q & 0x80 else 0)
        q &= 0xFF  
        
        #compute the affine transformation
        xformed = q ^ rotate(q, 1) ^ rotate(q, 2) ^ rotate(q, 3) ^ rotate(q, 4)
        
        SBOX[p] = xformed ^ 0x63
        
        if(p == 1):
            break
        
    #special case for zero
    SBOX[0] = 0x63
    return SBOX


def create_ISBOX(SBOX):
    ISBOX = [0] * 256
    for i in range(256):
        ISBOX[SBOX[i]] = i  # Reverse mapping from SBOX
    return ISBOX

strings = {}  # List to store input, encryption, and unique IDs
next_id = 1  # Unique ID for each request
key = "thisisakey123456"

# AES constants, S-box, and inverse S-Box (for AES-128)
SBOX = create_SBOX()
ISBOX = create_ISBOX(SBOX)

Nb = 4  # Block size for AES (in 32-bit words)
Nk = 4  # Key length (in 32-bit words) for AES-128
Nr = 10 # Number of rounds for AES-128


def key_expansion(key):   #Expand the cipher key into the key schedule
    
    def sub_word(word):   #Apply the AES S-box to each byte in a word
        return [SBOX[byte] for byte in word]


    def rot_word(word):   #Perform a cyclic permutation on a 4-byte word
        return word[1:] + word[:1]


    RCON = [
        [0x01, 0x00, 0x00, 0x00],
        [0x02, 0x00, 0x00, 0x00],
        [0x04, 0x00, 0x00, 0x00],
        [0x08, 0x00, 0x00, 0x00],
        [0x10, 0x00, 0x00, 0x00],
        [0x20, 0x00, 0x00, 0x00],
        [0x40, 0x00, 0x00, 0x00],
        [0x80, 0x00, 0x00, 0x00],
        [0x1B, 0x00, 0x00, 0x00],
        [0x36, 0x00, 0x00, 0x00],
    ]
    
    #The first Nk words are filled with the cypher key
    w = [list(key[4 * i: 4 * (i + 1)]) for i in range(Nk)]
    
    for i in range(Nk, Nb * (Nr + 1)):
        temp = w[i - 1][:]  # Ensure a copy is made
        
        if i % Nk == 0:
            temp = [a ^ b for a, b in zip(sub_word(rot_word(temp)), RCON[i // Nk - 1])]
        elif Nk > 6 and i % Nk == 4:
            temp = sub_word(temp)
        
        w.append([a ^ b for a, b in zip(w[i - Nk], temp)])
        
    return w


def add_round_key(state, round_key):
    return [[state[row][col] ^ round_key[col][row] for col in range(4)] for row in range(4)]


def sub_bytes(state):
    return [[SBOX[byte] for byte in row] for row in state]


def inv_sub_bytes(state):
    return [[ISBOX[byte] for byte in row] for row in state]

        
def shift_rows(state):
    return [
        state[0],
        state[1][1:] + state[1][:1],
        state[2][2:] + state[2][:2],
        state[3][3:] + state[3][:3]
    ]

def inv_shift_rows(state):
    return [
        state[0],
        state[1][-1:] + state[1][:-1],
        state[2][-2:] + state[2][:-2],
        state[3][-3:] + state[3][:-3]
    ]



def gmul(a, b):
        p = 0
        for _ in range(8):
            if b & 1:
                p ^= a
            hi_bit_set = a & 0x80
            a = (a << 1) & 0xFF
            if hi_bit_set:
                a ^= 0x1B
            b >>= 1
        return p
    
    
def mix_columns(state):
    for j in range(4):
        col = [state[i][j] for i in range(4)]
            
        new_col = [
            gmul(col[0], 2) ^ gmul(col[1], 3) ^ gmul(col[2], 1) ^ gmul(col[3], 1),
            gmul(col[0], 1) ^ gmul(col[1], 2) ^ gmul(col[2], 3) ^ gmul(col[3], 1),
            gmul(col[0], 1) ^ gmul(col[1], 1) ^ gmul(col[2], 2) ^ gmul(col[3], 3),
            gmul(col[0], 3) ^ gmul(col[1], 1) ^ gmul(col[2], 1) ^ gmul(col[3], 2)
        ]

        for i in range(4):
            state[i][j] = new_col[i]

    return state


def inv_mix_columns(state):
    
    for j in range(4):
        col = [state[i][j] for i in range(4)]
            
        new_col = [
            gmul(col[0], 14) ^ gmul(col[1], 11) ^ gmul(col[2], 13) ^ gmul(col[3], 9),
            gmul(col[0], 9) ^ gmul(col[1], 14) ^ gmul(col[2], 11) ^ gmul(col[3], 13),
            gmul(col[0], 13) ^ gmul(col[1], 9) ^ gmul(col[2], 14) ^ gmul(col[3], 11),
            gmul(col[0], 11) ^ gmul(col[1], 13) ^ gmul(col[2], 9) ^ gmul(col[3], 14)
        ]
        
        for i in range(4):
            state[i][j] = new_col[i]
        

    return state




def cipher(plaintext_block, round_keys):  
    # Convert ciphertext into state matrix (4x4)
    state = [[plaintext_block[row + 4 * col] for col in range(4)] for row in range(4)] 
    state = add_round_key(state, round_keys[0])
  
    for round_ in range(1, Nr):
        state = sub_bytes(state)    
        state = shift_rows(state)   
        state = mix_columns(state)      
        state = add_round_key(state, round_keys[round_])


    state = sub_bytes(state)    
    state = shift_rows(state)  
    state = add_round_key(state, round_keys[Nr])

    return [state[row][col] for col in range(4) for row in range(4)]






def inverse_cipher(ciphertext_block, round_keys):
    # Convert ciphertext into state matrix (4x4)
    state = [[ciphertext_block[row + 4 * col] for col in range(4)] for row in range(4)]
    state = add_round_key(state, round_keys[Nr]) #initial round key addition

    # Loop through rounds in reverse (decryption)     
    for round_ in range(Nr - 1, 0, -1):
        state = inv_shift_rows(state)
        state = inv_sub_bytes(state)
        state = add_round_key(state, round_keys[round_])
        state = inv_mix_columns(state)

    state = inv_shift_rows(state)
    state = inv_sub_bytes(state)
    state = add_round_key(state, round_keys[0])

    return [state[row][col] for col in range(4) for row in range(4)]



# Pad plaintext to 16 bytes if it's not a multiple of 16
def pad(plaintext):
    return plaintext + (16 - len(plaintext) % 16) * '\x00'

# Remove padding after decryption
def unpad(plaintext):
    return plaintext.rstrip('\x00')




#main function called to initiate encryption process
def aes_encrypt(plaintext, key):
    assert len(key) == 16, "Key must be 16 bytes for AES-128." #128 bit key (16 bytes = 128 bits)
    
    #128 bit blocks (16 bytes = 128 bits)
    plaintext = pad(plaintext) # Padding block if necessary
    plaintext_block = [ord(char) for char in plaintext] 
    
    round_keys = key_expansion([ord(c) for c in key]) #convert key to array of bytes before expansion   
    ciphertext_block = cipher(plaintext_block, [round_keys[i:i+4] for i in range(0, len(round_keys), 4)]) #convert round keys from flat 4-byte words to list of 4x4 matrices
 
    return ''.join(f'{x:02x}' for x in ciphertext_block) #return hex format



#main function to initiate decryption process
def aes_decrypt(encrypted_text, key):
    encrypted_text_block = bytes.fromhex(encrypted_text)
    round_keys = key_expansion([ord(c) for c in key]) #convert key to array of bytes before expansion
    decrypted_block = inverse_cipher(encrypted_text_block, [round_keys[i:i+4] for i in range(0, len(round_keys), 4)]) #convert round keys from flat 4-byte words to list of 4x4 matrices
    decrypted_text = ''.join(chr(byte) for byte in decrypted_block)
    return unpad(decrypted_text)



#Update Javascript in HTML file
@app.get('/get_strings')
def get_strings():
    response.content_type = 'application/json'
    return json.dumps(strings)  # Send stored strings as JSON

# Route to serve static CSS files
@app.route('/static/<filename>')
def serve_static(filename):
    response.content_type = 'text/css'  # Force CSS MIME type
    return static_file(filename, root='./static')


#Route to render images
@app.route('/static/images/<picture>')
def serve_pictures(picture):
    return static_file(picture, root='./static/images')


# Route to render the HOME page
@app.get('/')
def home():
    return template("home", strings=strings, time=int(time()))

# Route to render the ABOUT page
@app.get('/about')
def about():
    return template("about", strings=strings, time=int(time()))

# Route to render the LEARN page
@app.get('/learn')
def learn():
    return template("learn", strings=strings, time=int(time()))

# Route to render the TIMING page
@app.get('/timing')
def timing():
    return template("timing", strings=strings, time=int(time()))

# Route to render the HISTORY page
@app.get('/history')
def history():
    return template("history", strings=strings, time=int(time()))

# Route to render the DEMO page
@app.get('/demo')
def demo():
    return template("demo", strings=strings, time=int(time()))



#Accept input & send to encryption
@app.post('/encrypt')
def encrypt():
    global next_id

    # Get user input
    input_value = request.forms.get('input')

    # Ensure input is valid
    if not input_value:
        return "<p>Error: Text input is required.</p><a href='/'>Go Back</a>"

    # Encrypt the input
    input_encrypt = aes_encrypt(input_value, key)

    # Store the encrypted result
    strings[next_id] = {"id": next_id, "value": input_value, "encrypt": input_encrypt}
    next_id += 1
    

    # Redirect back to the landing page
    redirect('/demo')

    
    
#Decrypt entries on button push
@app.post('/decrypt/<entry_id>')
def decrypt(entry_id):
    entry_id = int(entry_id)  # Convert to integer
    if entry_id in strings:
        encrypted_text = strings[entry_id]['encrypt']
        decrypted_text = aes_decrypt(encrypted_text, key)  # Call to decryption function
        
        response.content_type = 'application/json'
        return json.dumps({"decrypted": decrypted_text})

    response.status = 404
    return json.dumps({"error": "Entry not found"})


# Run the app
if __name__ == '__main__':
    run(app, host='localhost', port=8080, debug=True)

Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [09/Apr/2025 14:12:52] "GET / HTTP/1.1" 200 3833
127.0.0.1 - - [09/Apr/2025 14:12:52] "GET /static/style1.css?v=1744225972 HTTP/1.1" 200 3126
127.0.0.1 - - [09/Apr/2025 14:12:54] "GET /learn HTTP/1.1" 200 2949
127.0.0.1 - - [09/Apr/2025 14:12:54] "GET /static/style1.css?v=1744225974 HTTP/1.1" 200 3126
127.0.0.1 - - [09/Apr/2025 14:13:42] "GET /learn HTTP/1.1" 200 2948
127.0.0.1 - - [09/Apr/2025 14:13:42] "GET /static/style1.css?v=1744226022 HTTP/1.1" 200 4255
127.0.0.1 - - [09/Apr/2025 14:14:18] "GET /learn HTTP/1.1" 200 2948
127.0.0.1 - - [09/Apr/2025 14:14:18] "GET /static/style1.css?v=1744226058 HTTP/1.1" 200 3803
127.0.0.1 - - [09/Apr/2025 14:20:26] "GET /learn HTTP/1.1" 200 3280
127.0.0.1 - - [09/Apr/2025 14:20:26] "GET /static/style1.css?v=1744226426 HTTP/1.1" 200 3803
127.0.0.1 - - [09/Apr/2025 14:21:56] "GET /learn HTTP/1.1" 200 3432
127.0.0.1 - - 

127.0.0.1 - - [09/Apr/2025 14:32:10] "GET /learn HTTP/1.1" 200 3429
127.0.0.1 - - [09/Apr/2025 14:32:10] "GET /static/style1.css?v=1744227130 HTTP/1.1" 200 3459
127.0.0.1 - - [09/Apr/2025 14:32:27] "GET /learn HTTP/1.1" 200 3429
127.0.0.1 - - [09/Apr/2025 14:32:27] "GET /static/style1.css?v=1744227147 HTTP/1.1" 200 3494
127.0.0.1 - - [09/Apr/2025 14:32:40] "GET /learn HTTP/1.1" 200 3429
127.0.0.1 - - [09/Apr/2025 14:32:40] "GET /static/style1.css?v=1744227160 HTTP/1.1" 200 3499
127.0.0.1 - - [09/Apr/2025 14:32:50] "GET /learn HTTP/1.1" 200 3429
127.0.0.1 - - [09/Apr/2025 14:32:50] "GET /static/style1.css?v=1744227170 HTTP/1.1" 200 3518
127.0.0.1 - - [09/Apr/2025 14:32:56] "GET /demo HTTP/1.1" 200 5853
127.0.0.1 - - [09/Apr/2025 14:32:56] "GET /static/style1.css?v=1744227176 HTTP/1.1" 200 3518
127.0.0.1 - - [09/Apr/2025 14:32:56] "GET /get_strings HTTP/1.1" 200 2
127.0.0.1 - - [09/Apr/2025 14:32:57] "GET /learn HTTP/1.1" 200 3429
127.0.0.1 - - [09/Apr/2025 14:32:57] "GET /static/style1.